In [1]:
from keras.models import model_from_json
import os
import cv2
import glob
import h5py
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
import scipy.io as io
from PIL import Image
import numpy as np

2024-06-11 22:19:06.746776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_model():
    
    json_file = open('/Users/apple/Desktop/CSRnet-master/models/Model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("/Users/apple/Desktop/CSRnet-master/weights/model_A_weights.h5")
    return loaded_model


def create_img(path):
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225


    im = np.expand_dims(im,axis  = 0)
    return im

In [3]:
root = '/Users/apple/Desktop/CSRnet-master/ShanghaiTech/'

In [4]:
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

182


In [6]:


model = load_model()
name = []
y_true = []
y_pred = []


for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground_truth') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    num = np.sum(model.predict(img))
    y_pred.append(np.sum(num))

    
data = pd.DataFrame({'name': name,'y_pred': y_pred,'y_true': y_true})
data.to_csv('CSV/A_on_A_test.csv', sep=',')

1/1 [==============================] - 4s 4s/step


In [7]:
data = pd.read_csv('CSV/A_on_A_test.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [8]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))
r_squared = r2_score(np.array(y_true), np.array(y_pred))

In [9]:
print("MAE : " , ans )
print("R Square:",r_squared)

MAE :  65.9119583021978
R Square: 0.911571673599671


In [11]:
data = pd.read_csv('CSV/B_on_B_test.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [12]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))
r_squared = r2_score(np.array(y_true), np.array(y_pred))

In [13]:
print("MAE : " , ans )
print("R Squared",r_squared)

MAE :  11.128531218354432
R Squared 0.9697467522614405


In [14]:
#Importing Libraries
import os
import glob
import h5py
import pandas as pd
from PIL import Image
import numpy as np
from keras.models import model_from_json
from sklearn.metrics import mean_absolute_error, r2_score

#Loading the model and weights
def load_model(model_path, weights_path):
    with open(model_path, 'r') as json_file:
        loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)
    return loaded_model

#Image normalization
def create_img(path):
    im = Image.open(path).convert('RGB')
    im = np.array(im)
    im = im / 255.0
    im[:, :, 0] = (im[:, :, 0] - 0.485) / 0.229
    im[:, :, 1] = (im[:, :, 1] - 0.456) / 0.224
    im[:, :, 2] = (im[:, :, 2] - 0.406) / 0.225
    im = np.expand_dims(im, axis=0)
    return im

#Loading the grount truth values for images, performing predictions using the model and storing predicted values
def process_images(image_paths, model):
    names, y_true, y_pred = [], [], []
    for image in image_paths:
        names.append(image)
        gt_file = image.replace('.jpg', '.h5').replace('images', 'ground_truth')
        with h5py.File(gt_file, 'r') as gt:
            groundtruth = np.asarray(gt['density'])
            true_count = np.sum(groundtruth)
        y_true.append(true_count)
        img = create_img(image)
        predicted_count = np.sum(model.predict(img))
        y_pred.append(predicted_count)
    return pd.DataFrame({'name': names, 'y_pred': y_pred, 'y_true': y_true})

# Paths and models setup
root = '/Users/apple/Desktop/CSRnet-master/'
models_info = {
    'A': {'test_path': os.path.join(root, 'ShanghaiTech/part_A_final/test_data', 'images'),
          'model_path': '/Users/apple/Desktop/CSRnet-master/models/Model.json', 'weights_path': '/Users/apple/Desktop/CSRnet-master/weights/model_A_weights.h5'},
    'B': {'test_path': os.path.join(root, 'ShanghaiTech/part_B_final/test_data', 'images'),
          'model_path': '/Users/apple/Desktop/CSRnet-master/models/Model.json', 'weights_path': '/Users/apple/Desktop/CSRnet-master/weights/model_B_weights.h5'}
}

#Storing all image paths into a single dataframe
all_data = pd.DataFrame()


for part in ['A', 'B']:
    info = models_info[part]
    model = load_model(info['model_path'], info['weights_path'])
    img_paths = glob.glob(os.path.join(info['test_path'], '*.jpg'))
    data = process_images(img_paths, model)
    data.to_csv(f'CSV/{part}_on_{part}_test.csv', sep=',')
    all_data = pd.concat([all_data, data])

# Calculate metrics
y_true = all_data['y_true']
y_pred = all_data['y_pred']
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae}")
print(f"R2 Score: {r2}")

1/1 [==============================] - 6s 6s/step
MAE: 31.149784088134766
R2 Score: 0.9428482592955035
